# BAUS Contingency Plan validation Tableau preprocessing

This notebook produces two CSV tables for subsequent use with the BAUS Contingency Plan validation Tableau workbook:
1. A pure long-format table for superdistrict/county-level comparisons of simple headcounts (e.g. `TOTHH` or `RETEMPN`)
2. A mixed-format table (source/year/TAZ on the rows, certain values on the columns) for superdistrict/county-level comparisons of residential and non-residential vacancy rates

In [1]:
import pandas as pd
from pathlib import Path
import getpass

user = getpass.getuser()

In [2]:
# First ingest Census data

census_dir = Path(r'X:\petrale\applications\travel_model_lu_inputs')

melt_vars = [
    'TOTHH',
    'TOTEMP',
    'RES_UNITS',
    'HHINCQ1',
    'HHINCQ2',
    'HHINCQ3',
    'HHINCQ4',
    'AGREMPN',
    'FPSEMPN',
    'HEREMPN',
    'MWTEMPN',
    'OTHEMPN',
    'RETEMPN',
]

dfs = []
vacancy_dfs = []
for year in ['2010', '2020', '2023']:
    in_df = pd.read_csv(census_dir / year / f'TAZ1454 {year} Land Use.csv')
    
    # 2010 has RES_UNITS already but the other years do not
    if 'RES_UNITS' not in in_df.columns:
        in_df['RES_UNITS'] = in_df['SFDU'] + in_df['MFDU']
    
    # Melt only those columns that are present in this year
    long = in_df.melt(
        id_vars='ZONE',
        value_vars=[var for var in melt_vars if var in in_df.columns]
    )

    long['source'] = 'Census'
    long['year'] = year

    dfs.append(long)

    # Store some data differently for vacancy purposes
    wide = in_df[['ZONE', 'TOTHH', 'RES_UNITS']]
    wide.columns = ['TAZ', 'TOTHH', 'residential_units']
    wide['residential_vacancy'] = 1 - wide['TOTHH'] / wide['residential_units']
    del wide['TOTHH']
    wide['source'] = 'Census/CoStar'
    wide['year'] = year
    vacancy_dfs.append(wide)    


C:\Users\dlevitt\AppData\Local\Temp\ipykernel_14148\2209976290.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide['residential_vacancy'] = 1 - wide['TOTHH'] / wide['residential_units']
C:\Users\dlevitt\AppData\Local\Temp\ipykernel_14148\2209976290.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wide['source'] = 'Census/CoStar'
C:\Users\dlevitt\AppData\Local\Temp\ipykernel_14148\2209976290.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [3]:
# Next ingest model run summaries from a variety of file locations

scenarios = {
    'PBA50': {
        'path': Path(rf"C:\Users\{user}\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Final Blueprint runs\Final Blueprint (s24)\BAUS v2.25 - FINAL VERSION"),
        'pattern': '*_taz_summaries_*'
    },
    'v0: PBA50 equivalent inputs': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v0"),
        'pattern': 'taz1_summary_*'
    },
    'v1: BASIS buildings in dev pipeline': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v1"),
        'pattern': 'taz1_summary_*'
    },
    'v2: BASIS buildings, updated control totals': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v2"),
        'pattern': 'taz1_summary_*'
    },
    'v3: aligned 2020 HHINC control totals': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v3"),
        'pattern': '*_taz1_summary_*'
    },
    'v4: aligned 2020 HHINC+employment control totals': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v4"),
        'pattern': '*_taz1_summary_*'
    },
    'v4: aligned 2020 HHINC+employment control totals': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v4"),
        'pattern': '*_taz1_summary_*'
    },
    'v5: aligned control totals, updated pipeline': {
        'path': Path(r"\\lumodel3\LUModel3Share\baus_main_current_PBA50_inputs\outputs\pba50_fbp_pr319_v5"),
        'pattern': '*_taz1_summary_*'
    },
}

for scenario, params in scenarios.items():
    # We need to handle PBA50 differently from the Contingency Plan runs
    if (params['path'] / 'travel_model_summaries').exists():
        long_dir = params['path'] / 'travel_model_summaries'
    else:  # This is the case for PBA50
        long_dir = params['path']
    
    for file in long_dir.glob(params['pattern']):
        if file.stem[-4:] in ['2010', '2020', '2025', '2035', '2050']:
            wide = pd.read_csv(file)
            long = wide.melt(
                id_vars='ZONE',
                value_vars=melt_vars
            )

            long['source'] = scenario
            long['year'] = file.stem[-4:]

            dfs.append(long)

    # Handle vacancy data
    if (params['path'] / 'core_summaries').exists():
        for file in (params['path'] / 'core_summaries').glob('*_interim_zone_output_*'):
            if file.stem[-4:] in ['2010', '2020', '2025', '2035', '2050']:
                vac_df = pd.read_csv(file, usecols=[
                    'TAZ',
                    'residential_units',
                    'residential_vacancy',
                    'non_residential_sqft',
                    'non_residential_vacancy',
                ])

                vac_df['source'] = scenario
                vac_df['year'] = file.stem[-4:]

                vacancy_dfs.append(vac_df)


df = pd.concat(dfs)
vacancy_df = pd.concat(vacancy_dfs)

df


,ZONE,variable,value,source,year
0,1,TOTHH,25.0,Census,2010
1,2,TOTHH,135.0,Census,2010
2,3,TOTHH,270.0,Census,2010
3,4,TOTHH,58.0,Census,2010
4,5,TOTHH,524.0,Census,2010
...,...,...,...,...,...
18897,1450,RETEMPN,463.0,"v5: aligned control totals, updated pipeline",2050
18898,1451,RETEMPN,113.0,"v5: aligned control totals, updated pipeline",2050
18899,1452,RETEMPN,137.0,"v5: aligned control totals, updated pipeline",2050
18900,1453,RETEMPN,3.0,"v5: aligned control totals, updated pipeline",2050


In [4]:
costar = pd.read_csv('costar_2020_taz1454.csv', dtype={'TAZ': int}, index_col='TAZ')
costar

,non_residential_sqft,non_residential_vacancy
TAZ,,
1,5945278.0,0.100360
2,9943897.0,0.104197
3,1173114.0,0.053345
4,6432617.0,0.224266
5,6368810.0,0.080895
...,...,...
1450,201497.0,0.031211
1451,78786.0,0.020943
1452,448024.0,0.044551


In [5]:
vacancy_df = vacancy_df.set_index('TAZ')
vacancy_df

,residential_units,residential_vacancy,source,year,non_residential_sqft,non_residential_vacancy
TAZ,,,,,,
1,61.0,0.590164,Census/CoStar,2010,NaN,NaN
2,152.0,0.111842,Census/CoStar,2010,NaN,NaN
3,294.0,0.081633,Census/CoStar,2010,NaN,NaN
4,174.0,0.666667,Census/CoStar,2010,NaN,NaN
5,735.0,0.287075,Census/CoStar,2010,NaN,NaN
...,...,...,...,...,...,...
1450,3057.0,0.341184,"v5: aligned control totals, updated pipeline",2050,689506.0,0.334477
1451,2110.0,0.277725,"v5: aligned control totals, updated pipeline",2050,155627.0,0.156010
1452,2308.0,0.296360,"v5: aligned control totals, updated pipeline",2050,931434.0,0.090985


In [6]:
vacancy_df.loc[(vacancy_df['source'] == 'Census/CoStar') & (vacancy_df['year'] == '2020') & vacancy_df.index.isin(costar.index), ['non_residential_sqft', 'non_residential_vacancy']] = costar
vacancy_df[vacancy_df['year'] == '2020']

,residential_units,residential_vacancy,source,year,non_residential_sqft,non_residential_vacancy
TAZ,,,,,,
1,48.0,0.000000,Census/CoStar,2020,5945278.0,0.100360
2,150.0,0.000000,Census/CoStar,2020,9943897.0,0.104197
3,377.0,0.299735,Census/CoStar,2020,1173114.0,0.053345
4,181.0,0.132597,Census/CoStar,2020,6432617.0,0.224266
5,618.0,0.226537,Census/CoStar,2020,6368810.0,0.080895
...,...,...,...,...,...,...
1450,2787.0,0.336204,"v5: aligned control totals, updated pipeline",2020,514027.0,0.246983
1451,2070.0,0.311594,"v5: aligned control totals, updated pipeline",2020,333428.0,0.105121
1452,2288.0,0.348776,"v5: aligned control totals, updated pipeline",2020,931434.0,0.227811


In [7]:
vacancy_df.to_csv('taz_data_wide_vacancy.csv')

In [8]:
# Linearly interpolate to generate 2023 estimates

for scenario in scenarios.keys():
    values_2020 = df.loc[(df['source'] == scenario) & (df['year'] == '2020'), 'value']
    values_2025 = df.loc[(df['source'] == scenario) & (df['year'] == '2025'), 'value']
    assert len(values_2020) == len(values_2025)
    values_2023 = values_2020 + (values_2025 - values_2020) * (3 / 5)

    # Construct 2023 records
    id_cols = df.loc[(df['source'] == scenario) & (df['year'] == '2020'), ['ZONE', 'variable']]
    df_2023 = pd.concat([id_cols, values_2023], axis=1)
    df_2023['source'] = scenario
    df_2023['year'] = '2023'
    
    dfs.append(df_2023)

In [9]:
out_df = pd.concat(dfs)

out_df.to_csv('taz_data_long.csv', index=False)

In [10]:
pd.crosstab(out_df['source'], out_df['year'])

year,2010,2020,2023,2025,2035,2050
source,,,,,,
Census,4362,18902,18902,0,0,0
PBA50,18902,18902,18902,18902,18902,18902
v0: PBA50 equivalent inputs,18902,18902,18902,18902,18902,18902
v1: BASIS buildings in dev pipeline,18902,18902,18902,18902,18902,18902
"v2: BASIS buildings, updated control totals",18902,18902,18902,18902,18902,18902
v3: aligned 2020 HHINC control totals,18902,18902,18902,18902,18902,18902
v4: aligned 2020 HHINC+employment control totals,18902,18902,18902,18902,18902,18902
"v5: aligned control totals, updated pipeline",18902,18902,18902,18902,18902,18902
